# Multi-Layer Perceptron

Importing packages and loading data:

In [32]:
using CSV, Random

iris = CSV.read("iris_data.csv")

# Mixing up the data
shuff = copy(iris)
iris = shuff[shuffle(1:end), :]

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,String⍰
1,5.9,3.0,4.2,1.5,versicolor
2,5.6,3.0,4.1,1.3,versicolor
3,7.1,3.0,5.9,2.1,virginica
4,4.6,3.2,1.4,0.2,setosa
5,6.8,3.2,5.9,2.3,virginica
6,6.1,3.0,4.9,1.8,virginica
7,6.3,2.9,5.6,1.8,virginica
8,6.5,3.2,5.1,2.0,virginica
9,5.4,3.9,1.7,0.4,setosa


Organizing the data:

In [33]:
# Making empty matrix X to append variable values (width/length, pedal/sepal)
X = zeros(4, 150)

# Making empty matrix Y to hold the value of which species it is
Y = zeros(3, 150)

# Filling in the matrices
for i = 1:150
    for j = 1:4
        X[j, i] = iris[i, j]
        # Determing species ([1,0,0]=setosa, [0,1,0]=versicolor, [0,0,1]=virginica)
        if iris[i, 5] == "setosa"
            Y[1, i] = 1.0
        elseif iris[i, 5] == "versicolor"
            Y[2, i] = 1.0
        else 
            Y[3, i] = 1.0
        end
    end
end


Looking at the data that determines species:

In [34]:
X

4×150 Array{Float64,2}:
 5.9  5.6  7.1  4.6  6.8  6.1  6.3  6.5  …  5.8  4.4  6.7  6.6  5.5  4.8  5.7
 3.0  3.0  3.0  3.2  3.2  3.0  2.9  3.2     2.6  2.9  3.1  2.9  4.2  3.0  2.8
 4.2  4.1  5.9  1.4  5.9  4.9  5.6  5.1     4.0  1.4  4.7  4.6  1.4  1.4  4.5
 1.5  1.3  2.1  0.2  2.3  1.8  1.8  2.0     1.2  0.2  1.5  1.3  0.2  0.1  1.3

Looking at the values of the species ($[1,0,0]^T=setosa, [0,1,0]^T=versicolor, [0,0,1]^T=virginica$):

In [35]:
Y

3×150 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  1.0  1.0  0.0
 1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  1.0  1.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

Creating the sigmoid function:
    $$\sigma(s)= \frac{1}{1+e^{-s}}$$

In [36]:
σ(s) = 1/(1+exp(-s))

σ (generic function with 1 method)

Taking the derivative of the Sigmoid function:
$$\frac{\delta \sigma(x)}{\delta(x)}=\sigma(x)*(1-\sigma(x))$$

In [37]:
dσ(s) = σ(s)*(1 - σ(s))

dσ (generic function with 1 method)

Defining the softmax function:
$$\sigma(a)_i= \frac{e^{a_i}}{\sum_{j=1}^K e^{a_j}}$$

In [40]:
softmax(a, i) = exp(a[i]) / (sum(exp(a[j]) for j = 1:length(a)))

softmax (generic function with 1 method)

Cross-entropy loss function:
$$H(p,q)=-\sum_{x \in X}p(x) \log q(x)$$

In [41]:
L(O, y) = -sum(y[i]*log(O[i]) for i = 1:length(y))

L (generic function with 1 method)

Hadamard product:
$$(A \circ B)_{ij}$$

In [42]:
hadamard(x, y) = [x[i]*y[i] for i = 1:length(x)]

hadamard (generic function with 1 method)

Defining our forward-propagation function:

In [43]:
function forward_propagation(x, y, W, b)
    a1 = copy(x)
    z2 = W[1]*a1 + b[1]
    a2 = σ.(z2)
    
    z3 = W[2]*a2 + b[2]
    a3 = σ.(z3)
    
    z4 = W[3]*a3 + b[3]
    a4 = σ.(z4)
    
    a = [a1, a2, a3, a4]
    z = [[0,0], z2, z3, z4]
    O = [softmax(a4, i) for i = 1:length(a4)]
    loss = L(O, y)
    return a, z, O, loss
end

    

forward_propagation (generic function with 1 method)

Defining the back-propagation function:

In [45]:
function back_propagation(x, y, W, b)
    a, z, O, loss = forward_propagation(x, y, W, b)
    δ4 = a[4] - y
    δ3 = hadamard(W[3]'*δ4, dσ.(z[3]))
    δ2 = hadamard(W[2]'*δ3, dσ.(z[2]))
    δ = [[0,0], δ2, δ3, δ4]
    return a, δ
end

function ∇L(x, y, W, b)
    
    a, δ = back_propagation(x, y, W, b)
    
    db1 = copy(δ[2])
    db2 = copy(δ[3])
    db3 = copy(δ[4])
    
    dW1 = δ[2]*a[1]'
    dW2 = δ[3]*a[2]'
    dW3 = δ[4]*a[3]'
    return [db1, db2, db3], [dW1, dW2, dW3]
end

function gradient_descent!(x, y, W, b, α)
    db, dW = ∇L(x, y, W, b)
    for i = 1:length(W)
        W[i] -= α*dW[i]
        b[i] -= α*b[i]
    end
end

    

gradient_descent! (generic function with 1 method)

Initializing weight matrix:

In [46]:
W1 = rand(5, 4)
W2 = rand(5, 5)
W3 = rand(3, 5)
W = [W1, W2, W3]

3-element Array{Array{Float64,2},1}:
 [0.633555 0.795808 0.917386 0.85992; 0.506258 0.214103 0.117201 0.338488; … ; 0.536134 0.216063 0.560318 0.441008; 0.793006 0.43241 0.242829 0.948146]           
 [0.93937 0.141083 … 0.729494 0.664136; 0.526831 0.249844 … 0.581573 0.688685; … ; 0.712343 0.985957 … 0.19891 0.237783; 0.0651416 0.506037 … 0.00524659 0.389204]
 [0.765946 0.669653 … 0.621225 0.489132; 0.359094 0.928174 … 0.748754 0.359898; 0.00272376 0.266011 … 0.357457 0.163997]                                          

Initializing bias matrix:

In [47]:
b1 = -1*ones(5)
b2 = -1*ones(5)
b3 = -1*ones(3)
b = [b1, b2, b3]

3-element Array{Array{Float64,1},1}:
 [-1.0, -1.0, -1.0, -1.0, -1.0]
 [-1.0, -1.0, -1.0, -1.0, -1.0]
 [-1.0, -1.0, -1.0]            

In [48]:
for _ in 1:1000000
    j = rand(1:50)
    gradient_descent!(X[:,j], Y[:,j], W, b, 0.37) # The last number is alpha which is our step length
end

Testing our forward_propagation function:

In [49]:
forward_propagation(X[:,110], Y[:,110], W, b)[3:4]

([0.320118, 0.330287, 0.349595], 1.1077925290535202)

Checking the result:

In [50]:
Y[:, 20]

3-element Array{Float64,1}:
 0.0
 1.0
 0.0

Another way to randomize the data:

In [51]:
train_data = zeros(4, 100)
train_label = zeros(3, 100)

for i in 1:100
    j = rand(1:3)
    if j == 1
        k = rand(1:50)
        train_data[:, i] = copy(X[:, k])
        train_label[:, i] = copy(Y[:,k])
    elseif j == 2
        k = rand(50:100)
        train_data[:, i] = copy(X[:, k])
        train_label[:, i] = copy(Y[:,k])
    else
        k = rand(100:150)
        train_data[:, i] = copy(X[:, k])
        train_label[:, i] = copy(Y[:,k])
    end
end


Defining the Mini-Batch Gradient Descent:

In [52]:
function mini_batch_∇L(train_data, train_label, W, b, m)
    
    i = rand(1:100)
    a, δ = back_propagation(train_data[:, i], train_label[:,i], W, b)
    
    db1 = (δ[2])
    db2 = (δ[3])
    db3 = (δ[4])
    
    dW1 = δ[2]*a[1]'
    dW2 = δ[3]*a[2]'
    dW3 = δ[4]*a[3]'
    
    for _ in 1:m
        j = rand(1:100)
        a, δ = back_propagation(train_data[:, j], train_label[:,j], W, b)
        
        db1 += copy(δ[2])
        db2 += copy(δ[3])
        db3 += copy(δ[4])
    
        dW1 += δ[2]*a[1]'
        dW2 += δ[3]*a[2]'
        dW3 += δ[4]*a[3]'
    end
    
    return [db1/m, db2/m, db3/m], [dW1/m, dW2/m, dW3/m]
end

mini_batch_∇L (generic function with 1 method)

Calculating the error and updating the model:

In [55]:
function stochastic_gradient_descent!(train_data, train_label, W, b, α, m)
    db, dW = mini_batch_∇L(train_data, train_label, W, b, m)

    for i = 1:length(W)
        W[i] -= α*dW[i]
        b[i] -= α*b[i]
    end
end


stochastic_gradient_descent! (generic function with 1 method)

In [56]:
# # Initialize weight matrices
# W1 = rand(5, 4)
# W2 = rand(5, 5)
# W3 = rand(3, 5)
# W = [W1, W2, W3]

# # Initialize the bias
# b1 = -1*ones(5)
# b2 = -1*ones(5)
# b3 = -1*ones(3)
# b = [b1, b2, b3]

In [57]:
for _ in 1:1000000
    stochastic_gradient_descent!(train_data, train_label, W, b, 0.38, 23)
end

In [58]:
forward_propagation(X[:,86], Y[:,145], W, b)[3:4]

([0.308698, 0.331472, 0.359831], 1.1753924827844906)

In [59]:
using MLDatasets, Images, TestImages

In [61]:
train_x, train_y = MNIST.traindata()
test_x, test_y = MNIST.testdata();

In [62]:
X = []
Y = []

for i = 1:60000
    push!(X, reshape(train_x[:, :, i], 784))
    y = zeros(10)
    y[train_y[i]+1] = 1
    push!(Y, y)
end

In [63]:
train_x = zeros(784, 60000)
train_y = zeros(10, 60000)

for i = 1:60000
    train_x[:, 1] = X[i]
    train_y[:, i] = Y[i]
end

In [66]:
for _ in 1:10000
    stochastic_gradient_descent!(train_data, train_label, W, b, 0.30, 23)
end